In [2]:
# Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from lightgbm import LGBMRegressor
import warnings

# Configuración de gráficos
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({'font.size':8})
warnings.filterwarnings('once')

# Descargar y preparar los datos
from skforecast.datasets import fetch_dataset
datos = fetch_dataset(name='vic_electricity', raw=True)
datos['Time'] = pd.to_datetime(datos['Time'], format='%Y-%m-%dT%H:%M:%SZ')
datos = datos.set_index('Time')
datos = datos.asfreq('30min')
datos = datos.sort_index()

# Verificar si el índice temporal está completo
assert (datos.index == pd.date_range(start=datos.index.min(),
                                     end=datos.index.max(),
                                     freq=datos.index.freq)).all(), "El índice temporal no está completo"

# Agregar datos a nivel horario
datos = datos.drop(columns='Date')
datos = datos.resample(rule='H', closed='left', label='right').mean()

# Separar en conjuntos de entrenamiento, validación y prueba
datos = datos.loc['2012-01-01 00:00:00': '2014-12-30 23:00:00']
fin_train = '2013-12-31 23:59:00'
fin_validacion = '2014-11-30 23:59:00'
datos_train = datos.loc[:fin_train, :].copy()
datos_val = datos.loc[fin_train:fin_validacion, :].copy()
datos_test = datos.loc[fin_validacion:, :].copy()

# Exploración gráfica
fig = go.Figure()
fig.add_trace(go.Scatter(x=datos_train.index, y=datos_train['Demand'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=datos_val.index, y=datos_val['Demand'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(x=datos_test.index, y=datos_test['Demand'], mode='lines', name='Test'))
fig.update_layout(
    title='Demanda eléctrica horaria',
    xaxis_title="Fecha",
    yaxis_title="Demanda (MWh)",
    legend_title="Partición:",
    width=850,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1,
        xanchor="left",
        x=0.001
    )
)
fig.show()

# Modelo baseline
forecaster_baseline = ForecasterEquivalentDate(
    offset=pd.DateOffset(days=1),
    n_offsets=1
)

forecaster_baseline.fit(y=datos.loc[:fin_validacion, 'Demand'])

# Backtesting modelo baseline
metric_baseline, predictions_baseline = backtesting_forecaster(
    forecaster=forecaster_baseline,
    y=datos['Demand'],
    steps=24,
    metric='mean_absolute_error',
    initial_train_size=len(datos.loc[:fin_validacion]),
    refit=False,
    n_jobs='auto',
    verbose=False,
    show_progress=True
)

print(f"Backtest error (MAE) del modelo baseline: {metric_baseline}")

# Modelo autoregresivo recursivo con LGBMRegressor
forecaster_autoreg = ForecasterAutoreg(
    regressor=LGBMRegressor(random_state=15926, verbose=-1),
    lags=24
)
forecaster_autoreg.fit(y=datos.loc[:fin_validacion, 'Demand'])

# Backtesting modelo autoregresivo recursivo
metric_autoreg, predictions_autoreg = backtesting_forecaster(
    forecaster=forecaster_autoreg,
    y=datos['Demand'],
    steps=24,
    metric='mean_absolute_error',
    initial_train_size=len(datos.loc[:fin_validacion]),
    refit=False,
    n_jobs='auto',
    verbose=True,
    show_progress=True
)

print(f"Backtest error (MAE) del modelo autoregresivo: {metric_autoreg}")

# Gráfico predicción vs valores reales para el modelo autoregresivo
fig = go.Figure()
trace1 = go.Scatter(x=datos_test.index, y=datos_test['Demand'], name="Valor real", mode="lines")
trace2 = go.Scatter(x=predictions_autoreg.index, y=predictions_autoreg['pred'], name="Predicción", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Predicción vs Valor real en datos de prueba (modelo autoregresivo)",
    xaxis_title="Fecha",
    yaxis_title="Demanda",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="left",
        x=0.001
    )
)
fig.show()


vic_electricity
---------------
Half-hourly electricity demand for Victoria, Australia
O'Hara-Wild M, Hyndman R, Wang E, Godahewa R (2022).tsibbledata: Diverse
Datasets for 'tsibble'. https://tsibbledata.tidyverts.org/,
https://github.com/tidyverts/tsibbledata/.
https://tsibbledata.tidyverts.org/reference/vic_elec.html
Shape of the dataset: (52608, 5)


C:\Users\arzua\AppData\Local\Temp\ipykernel_19160\2212318176.py:36: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



  0%|          | 0/30 [00:00<?, ?it/s]

Backtest error (MAE) del modelo baseline: 308.3752715958334


c:\Users\arzua\OneDrive\Escritorio\preelec\src\ml_elec_1\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning:

Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "c:\Users\arzua\OneDrive\Escritorio\preelec\src\ml_elec_1\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Information of backtesting process
----------------------------------
Number of observations used for initial training: 25560
Number of observations used for backtesting: 720
    Number of folds: 30
    Number of steps per fold: 24
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2012-01-01 00:00:00 -- 2014-11-30 23:00:00  (n=25560)
    Validation: 2014-12-01 00:00:00 -- 2014-12-01 23:00:00  (n=24)
Fold: 1
    Training:   2012-01-01 00:00:00 -- 2014-11-30 23:00:00  (n=25560)
    Validation: 2014-12-02 00:00:00 -- 2014-12-02 23:00:00  (n=24)
Fold: 2
    Training:   2012-01-01 00:00:00 -- 2014-11-30 23:00:00  (n=25560)
    Validation: 2014-12-03 00:00:00 -- 2014-12-03 23:00:00  (n=24)
Fold: 3
    Training:   2012-01-01 00:00:00 -- 2014-11-30 23:00:00  (n=25560)
    Validation: 2014-12-04 00:00:00 -- 2014-12-04 23:00:00  (n=24)
Fold: 4
    Training:   2012-01-01 00:00:00 -- 2014-11-30 23:00:00  (n=25560)
    Validation: 2014-12-05

  0%|          | 0/30 [00:00<?, ?it/s]

Backtest error (MAE) del modelo autoregresivo: 236.38849515592196


In [3]:
import pandas as pd

In [4]:
# Importar librerías adicionales
from astral.sun import sun
from astral import LocationInfo
from sklearn.preprocessing import PolynomialFeatures

# Variables basadas en el calendario
variables_calendario = pd.DataFrame(index=datos.index)
variables_calendario['mes'] = variables_calendario.index.month
variables_calendario['semana_anyo'] = variables_calendario.index.isocalendar().week
variables_calendario['dia_semana'] = variables_calendario.index.day_of_week + 1
variables_calendario['hora_dia'] = variables_calendario.index.hour + 1

# Variables basadas en la luz solar
location = LocationInfo("Melbourne", "Australia", latitude=-37.8, longitude=144.95, timezone='Australia/Melbourne')
hora_amanecer = [sun(location.observer, date=date, tzinfo=location.timezone)['sunrise'].hour for date in datos.index]
hora_anochecer = [sun(location.observer, date=date, tzinfo=location.timezone)['sunset'].hour for date in datos.index]

variables_solares = pd.DataFrame({
    'hora_amanecer': hora_amanecer,
    'hora_anochecer': hora_anochecer}, 
    index=datos.index
)
variables_solares['horas_luz_solar'] = variables_solares['hora_anochecer'] - variables_solares['hora_amanecer']
variables_solares['es_de_dia'] = np.where(
    (datos.index.hour >= variables_solares['hora_amanecer']) & 
    (datos.index.hour < variables_solares['hora_anochecer']),
    1,
    0
)

# Variables basadas en festivos
variables_festivos = datos[['Holiday']].astype(int)
variables_festivos['holiday_dia_anterior'] = variables_festivos['Holiday'].shift(24)
variables_festivos['holiday_dia_siguiente'] = variables_festivos['Holiday'].shift(-24)

# Variables meteorológicas
variables_temp = datos[['Temperature']].copy()
variables_temp['temp_roll_mean_1_dia'] = variables_temp['Temperature'].rolling(24, closed='left').mean()
variables_temp['temp_roll_mean_7_dia'] = variables_temp['Temperature'].rolling(24*7, closed='left').mean()
variables_temp['temp_roll_max_1_dia'] = variables_temp['Temperature'].rolling(24, closed='left').max()
variables_temp['temp_roll_min_1_dia'] = variables_temp['Temperature'].rolling(24, closed='left').min()
variables_temp['temp_roll_max_7_dia'] = variables_temp['Temperature'].rolling(24*7, closed='left').max()
variables_temp['temp_roll_min_7_dia'] = variables_temp['Temperature'].rolling(24*7, closed='left').min()

# Unir todas las variables exógenas
variables_exogenas = pd.concat([variables_calendario, variables_solares, variables_temp, variables_festivos], axis=1)
variables_exogenas = variables_exogenas.dropna()

# Codificación cíclica de las variables de calendario y luz solar
def codificacion_ciclica(datos: pd.Series, longitud_ciclo: int) -> pd.DataFrame:
    seno = np.sin(2 * np.pi * datos/longitud_ciclo)
    coseno = np.cos(2 * np.pi * datos/longitud_ciclo)
    return pd.DataFrame({f"{datos.name}_seno": seno, f"{datos.name}_coseno": coseno})

mes_encoded = codificacion_ciclica(variables_exogenas['mes'], longitud_ciclo=12)
semana_anyo_encoded = codificacion_ciclica(variables_exogenas['semana_anyo'], longitud_ciclo=52)
dia_semana_encoded = codificacion_ciclica(variables_exogenas['dia_semana'], longitud_ciclo=7)
hora_dia_encoded = codificacion_ciclica(variables_exogenas['hora_dia'], longitud_ciclo=24)
hora_amanecer_encoded = codificacion_ciclica(variables_exogenas['hora_amanecer'], longitud_ciclo=24)
hora_anochecer_encoded = codificacion_ciclica(variables_exogenas['hora_anochecer'], longitud_ciclo=24)

variables_ciclicas = pd.concat([mes_encoded, semana_anyo_encoded, dia_semana_encoded, hora_dia_encoded, hora_amanecer_encoded, hora_anochecer_encoded], axis=1)
variables_exogenas = pd.concat([variables_exogenas, variables_ciclicas], axis=1)

# Entrenar el modelo con variables exógenas
exog_features = variables_exogenas.columns.tolist()
datos = datos[['Demand']].merge(variables_exogenas, left_index=True, right_index=True, how='left')
datos = datos.dropna()
datos = datos.astype('float32')

# Separar en conjuntos de entrenamiento, validación y prueba con variables exógenas
datos_train = datos.loc[:fin_train, :].copy()
datos_val = datos.loc[fin_train:fin_validacion, :].copy()
datos_test = datos.loc[fin_validacion:, :].copy()

# Crear forecaster con variables exógenas
forecaster_exog = ForecasterAutoreg(
    regressor=LGBMRegressor(random_state=15926, verbose=-1),
    lags=24
)

forecaster_exog.fit(y=datos.loc[:fin_validacion, 'Demand'], exog=datos.loc[:fin_validacion, exog_features])

# Backtesting modelo autoregresivo recursivo con variables exógenas
metric_exog, predictions_exog = backtesting_forecaster(
    forecaster=forecaster_exog,
    y=datos['Demand'],
    exog=datos[exog_features],
    steps=24,
    metric='mean_absolute_error',
    initial_train_size=len(datos.loc[:fin_validacion]),
    refit=False,
    n_jobs='auto',
    verbose=True,
    show_progress=True
)

print(f"Backtest error (MAE) del modelo autoregresivo con variables exógenas: {metric_exog}")

# Gráfico predicción vs valores reales para el modelo con variables exógenas
fig = go.Figure()
trace1 = go.Scatter(x=datos_test.index, y=datos_test['Demand'], name="Valor real", mode="lines")
trace2 = go.Scatter(x=predictions_exog.index, y=predictions_exog['pred'], name="Predicción", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Predicción vs Valor real en datos de prueba (modelo con variables exógenas)",
    xaxis_title="Fecha",
    yaxis_title="Demanda",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="left",
        x=0.001
    )
)
fig.show()


Information of backtesting process
----------------------------------
Number of observations used for initial training: 25392
Number of observations used for backtesting: 696
    Number of folds: 29
    Number of steps per fold: 24
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2012-01-08 00:00:00 -- 2014-11-30 23:00:00  (n=25392)
    Validation: 2014-12-01 00:00:00 -- 2014-12-01 23:00:00  (n=24)
Fold: 1
    Training:   2012-01-08 00:00:00 -- 2014-11-30 23:00:00  (n=25392)
    Validation: 2014-12-02 00:00:00 -- 2014-12-02 23:00:00  (n=24)
Fold: 2
    Training:   2012-01-08 00:00:00 -- 2014-11-30 23:00:00  (n=25392)
    Validation: 2014-12-03 00:00:00 -- 2014-12-03 23:00:00  (n=24)
Fold: 3
    Training:   2012-01-08 00:00:00 -- 2014-11-30 23:00:00  (n=25392)
    Validation: 2014-12-04 00:00:00 -- 2014-12-04 23:00:00  (n=24)
Fold: 4
    Training:   2012-01-08 00:00:00 -- 2014-11-30 23:00:00  (n=25392)
    Validation: 2014-12-05

  0%|          | 0/29 [00:00<?, ?it/s]

Backtest error (MAE) del modelo autoregresivo con variables exógenas: 139.88307844536678


Modelos Evaluados

Modelo Baseline (ForecasterEquivalentDate)
    
Descripción: Este modelo de referencia predice cada hora utilizando el valor observado en la misma hora del día anterior.
        Error MAE (Mean Absolute Error): 308.38

Modelo Autoregresivo Recursivo (ForecasterAutoreg)

Descripción: Un modelo autoregresivo recursivo utilizando LGBMRegressor con una ventana temporal de 24 horas (24 lags) para predecir la demanda de la siguiente hora.
        Error MAE (Mean Absolute Error): 236.39

Modelo Autoregresivo Recursivo con Variables Exógenas (ForecasterAutoreg)

Descripción: Un modelo autoregresivo recursivo utilizando LGBMRegressor con una ventana temporal de 24 horas (24 lags) y variables exógenas adicionales (como datos de calendario, luz solar y temperatura) para predecir la demanda de la siguiente hora.
        Error MAE (Mean Absolute Error): 139.88
Conclusiones

    Modelo Baseline: Sirve como referencia y muestra un error MAE de 308.38.
    Modelo Autoregresivo Recursivo: Mejora significativamente el error MAE a 236.39 en comparación con el modelo baseline.
    Modelo Autoregresivo Recursivo con Variables Exógenas: Incorpora información adicional (como variables exógenas) y reduce aún más el error MAE a 139.88, demostrando una mejora considerable en la capacidad predictiva del modelo.

In [5]:
import joblib

# Directorio donde se guardarán los modelos
save_dir = 'C:\\Users\\arzua\\OneDrive\\Escritorio\\preelec\\src\\'

# Guardar el modelo baseline
joblib.dump(forecaster_baseline, save_dir + 'forecaster_baseline.pkl')

# Guardar el modelo autoregresivo recursivo
joblib.dump(forecaster_autoreg, save_dir + 'forecaster_autoreg.pkl')

# Guardar el modelo autoregresivo recursivo con variables exógenas
joblib.dump(forecaster_autoreg_exog, save_dir + 'forecaster_autoreg_exog.pkl')


NameError: name 'forecaster_autoreg_exog' is not defined